In [1]:
import os
import sys
import time
import joblib
import numpy as np
import cv2
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from keras.models import Sequential
from keras.layers import (
    Input, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense,
    LeakyReLU, ReLU
)
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

MODEL_DIR = os.path.join('..', '..', 'models')
DATA_ROOT_KAGGLE = os.path.join('..', '..', 'data_subsets', 'fire_dataset')
TEST_EVAL_DATA_ROOT = os.path.join('..', '..', 'data_subsets', 'test_kaggle')

def is_kaggle_image_fire(image_path):
    parent_dir = os.path.basename(os.path.dirname(image_path))
    return parent_dir == 'fire_images'

def load_prep_4_cnn(data_dir, target_size=(128, 128)):
    all_images = []
    all_labels = []
    class_dirs = {'fire_images': 1, 'non_fire_images': 0}

    for class_name, label in class_dirs.items():
        class_path = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_path): continue
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            if not img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')): continue
            try:
                img = cv2.imread(img_path)
                if img is None: continue
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img_resized = cv2.resize(img, target_size)
                img_normalized = img_resized.astype(np.float32) / 255.0
                all_images.append(img_normalized)
                all_labels.append(label)
            except Exception as e:
                continue
    return np.array(all_images), np.array(all_labels)

def load_all_artifacts(dataset_choice):
    print(f"\n--- Loading saved models for {dataset_choice} dataset ---")
    artifacts = {}
    
    # MLP Modeli ve CNN Özellik Çıkarıcı
    try:
        model_path_mlp = os.path.join(MODEL_DIR, 'Dfire_hybrid_custom_mlp_best_model_Scaled_All_CNN.keras')
        artifacts['mlp_model'] = tf.keras.models.load_model(model_path_mlp, compile=False)
        print(f"Loaded mlp_model from {model_path_mlp}")

        # MLP için özellik çıkarıcı olarak kullanılan CNN mimarisini YENİDEN OLUŞTURUN.
        # Bu, create_custom_cnn'nin varsayılan parametrelerinin 128x128x3 görüntüden
        # 32x32x64 => 65536 Flattened özellik ürettiği varsayımına dayanır.
        cnn_base_for_mlp_features = create_custom_cnn(
            input_shape=(128, 128, 3), # load_prep_4_cnn'nin hedef boyutu
            conv_blocks=((32, (3,3)), (64, (3,3))), # Bu, 65536 özelliği üreten varsayılan ayarlar.
            # dense_layers, dropout_rate, activation gibi diğer parametreler,
            # Flatten katmanına kadar olan kısmı oluşturduğumuz için burada önemli değil.
        )
        artifacts['cnn_feature_extractor_for_mlp'] = create_cnn_feature_extractor(cnn_base_for_mlp_features)
        print(f"Created CNN feature extractor for mlp_model (reconstructed based on expected input shape).")

        # MLP özelliklerini ölçeklemek için kullanılan scaler'ı yükle.
        # Eğer MLP eğitilirken özellikler ölçeklendiyse bu gereklidir.
        try:
            scaler_path_mlp = os.path.join(MODEL_DIR, 'scaler_for_Scaled_All_CNN.pkl') # <-- BU DOSYA ADINI KONTROL EDİN
                                                                                        # EĞER FARKLI BİR YERE KAYDETTİYSENİZ!
            artifacts['mlp_scaler'] = joblib.load(scaler_path_mlp)
            print(f"Loaded MLP scaler from {scaler_path_mlp}")
        except Exception as e:
            print(f"Could not load MLP scaler: {e}. MLP predictions might be inaccurate if scaling was applied during training but not here.")
            pass

    except Exception as e:
        print(f"Could not load mlp_model or create CNN feature extractor for MLP: {e}")
        pass

    # Diğer modeller (LightGBM, SVM, transformer_lightgbm_model) yüklenmesi
    # ... (Mevcut kodunuzu buraya kopyalayın)
    
    # transformer_lightgbm_model için SelectKBest ve LightGBM modelini ayrı ayrı yükleme
    # Önceki çözümlerde verilen transformer_lightgbm_model yükleme bloğunu buraya ekleyin.
    # Örneğin:
    try:
        selector_path = os.path.join(MODEL_DIR, 'Dfire_hybrid_transformer_Scaled_Corr75%_CNN.pkl') # Eğer SelectKBest bu dosyadaysa
        artifacts['transformer_selector'] = joblib.load(selector_path)
        print(f"Loaded transformer_selector (SelectKBest) from {selector_path}")

        classifier_path = os.path.join(MODEL_DIR, 'Dfire_hybrid_transformer_Scaled_Corr75%_CNN_LGBM.pkl') # LightGBM modelinizin kaydedildiği dosya
        artifacts['transformer_lgbm_classifier'] = joblib.load(classifier_path)
        print(f"Loaded transformer_lgbm_classifier from {classifier_path}")

    except Exception as e:
        print(f"Could not load transformer_lightgbm_model components: {e}")
        pass

    return artifacts

def fetch_original_test_split(data_root, target_size=(128, 128), test_size=0.2, random_state=42):
    print(f"\n--- Fetching original test split from '{data_root}' ---")
    all_images, all_labels = load_prep_4_cnn(data_root, target_size)

    if all_images.size == 0:
        print("No images loaded for test split.")
        return None, None

    if len(np.unique(all_labels)) < 2:
        print("Not enough classes for stratified split. Skipping test split generation.")
        return None, None

    try:
        X_train, X_test, y_train, y_test = train_test_split(
            all_images, all_labels, test_size=test_size, random_state=random_state, stratify=all_labels
        )
        print(f"Successfully recreated test split with {X_test.shape[0]} samples.")
        return X_test, y_test
    except ValueError as e:
        print(f"Error during train_test_split: {e}. Check if there's enough data or classes for stratification.")
        return None, None

def preprocess_image_for_cnn(image_path, artifacts, model_str, target_size=(128, 128)):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Warning: Could not read image {image_path}")
        return None

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, target_size)
    img_normalized = img_resized.astype(np.float32) / 255.0
    cnn_features = np.expand_dims(img_normalized, axis=0) # Şekil (1, H, W, C)
    return cnn_features


def evaluate_folder(folder_path, artifacts, model_str):
    print(f"\n--- Processing images in folder: {folder_path} using {model_str} ---")
    model = artifacts.get(model_str)
    if model is None:
        print(f"Model {model_str} not found in artifacts. Skipping folder evaluation.")
        return

    all_true_labels = []
    all_predictions = []
    processed_count = 0

    if not os.path.isdir(folder_path):
        print(f"Folder not found: {folder_path}")
        return

    image_paths = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(os.path.join(root, file))

    if not image_paths:
        print(f"No images found in {folder_path}")
        return

    for image_path in tqdm(image_paths, desc="Processing folder images"):
        base_dir_name = os.path.basename(os.path.dirname(image_path))
        true_label = 1 if 'fire_images' in base_dir_name.lower() else 0

        img_preprocessed = preprocess_image_for_cnn(image_path, artifacts, model_str)
        if img_preprocessed is None:
            continue

        try:
            # Düzeltme: mlp_model için de girdi düzleştirildi
            if model_str == 'mlp_model':
                mlp_model_instance = artifacts.get('mlp_model')
                cnn_feature_extractor_instance = artifacts.get('cnn_feature_extractor_for_mlp')
                mlp_scaler_instance = artifacts.get('mlp_scaler') # Scaler'ı al

                if mlp_model_instance is None or cnn_feature_extractor_instance is None:
                    print(f"MLP model or CNN feature extractor not loaded for {image_path}. Skipping.")
                    continue # veya return eğer tek görüntü işliyorsanız

                # Adım 1: Görüntüyü CNN özellik çıkarıcıdan geçirerek özellikleri elde et
                # img_preprocessed'ın şekli (1, 128, 128, 3) olmalı
                cnn_extracted_features = cnn_feature_extractor_instance.predict(img_preprocessed, verbose=0)
                # cnn_extracted_features'ın şekli şimdi (1, 65536) olacaktır

                # Adım 2: Elde edilen özellikleri ölçekle (eğer scaler yüklendiyse)
                if mlp_scaler_instance is not None:
                    cnn_extracted_features_scaled = mlp_scaler_instance.transform(cnn_extracted_features)
                else:
                    cnn_extracted_features_scaled = cnn_extracted_features

                # Adım 3: Ölçeklenmiş özellikleri MLP modeline ver
                prediction_proba = mlp_model_instance.predict(cnn_extracted_features_scaled, verbose=0)
                prediction = (prediction_proba > 0.5).astype(int)[0][0]
            else: # Scikit-learn tabanlı modeller (LightGBM, SVM, transformer_lightgbm_model)
                flat_features = img_preprocessed.reshape(img_preprocessed.shape[0], -1) # Şekil (1, H*W*C)
                if hasattr(model, 'predict_proba'):
                    prediction_proba = model.predict_proba(flat_features)
                    prediction = (prediction_proba[:, 1] > 0.5).astype(int)[0]
                else:
                    prediction = model.predict(flat_features)[0]

            all_true_labels.append(true_label)
            all_predictions.append(prediction)
            processed_count += 1
        except Exception as e:
            print(f"Error during prediction for {image_path} with {model_str}: {e}")
            continue

    print(f"\nProcessed {processed_count} images from the folder.")

    if not all_true_labels:
        print("No successful predictions for folder.")
        return

    y_true = np.array(all_true_labels)
    y_pred = np.array(all_predictions)

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    conf_matrix = confusion_matrix(y_true, y_pred)

    print(f"\n--- Performance for {model_str} on Folder Data ---")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall (Sensitivity): {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Confusion Matrix:\n{conf_matrix}")

def process_single_image(image_path, artifacts, model_str):
    print(f"\n--- Processing single image: {image_path} using {model_str} ---")
    model = artifacts.get(model_str)
    if model is None:
        print(f"Model {model_str} not found in artifacts. Skipping single image processing.")
        return

    img_display = cv2.imread(image_path)
    if img_display is None:
        print(f"Warning: Could not read image for display {image_path}")
        return

    base_dir_name = os.path.basename(os.path.dirname(image_path))
    true_label_text = 'Fire' if base_dir_name.lower() == 'fire_images' else 'Non-Fire'
    true_label_num = 1 if base_dir_name.lower() == 'fire_images' else 0

    img_preprocessed = preprocess_image_for_cnn(image_path, artifacts, model_str)
    if img_preprocessed is None:
        return

    prediction_text = "Unknown"
    try:
        # Düzeltme: mlp_model için de girdi düzleştirildi
        if model_str == 'mlp_model':
            flat_features = img_preprocessed.reshape(img_preprocessed.shape[0], -1)
            prediction_proba = model.predict(flat_features, verbose=0)
            prediction = (prediction_proba > 0.5).astype(int)[0][0]
        elif model_str == 'transformer_lightgbm_model': # <--- BURADA DEĞİŞİKLİK
                selector = artifacts.get('transformer_selector')
                classifier = artifacts.get('transformer_lgbm_classifier')
                if selector is None or classifier is None:
                    print(f"Transformer components not loaded for {image_path}. Skipping.")
                    pass

                flat_features = img_preprocessed.reshape(img_preprocessed.shape[0], -1)
                # Önce SelectKBest ile özellikleri dönüştür
                selected_features = selector.transform(flat_features)
        else:
            flat_features = img_preprocessed.reshape(img_preprocessed.shape[0], -1)
            if hasattr(model, 'predict_proba'):
                prediction_proba = model.predict_proba(flat_features)
                prediction = (prediction_proba[:, 1] > 0.5).astype(int)[0]
            else:
                prediction = model.predict(flat_features)[0]

        prediction_text = "Fire" if prediction == 1 else "Non-Fire"
        print(f"True Label: {true_label_text} ({true_label_num})")
        print(f"Prediction by {model_str}: {prediction_text}")
    except Exception as e:
        print(f"Error during prediction for {image_path} with {model_str}: {e}")
        return

    plt.imshow(cv2.cvtColor(img_display, cv2.COLOR_BGR2RGB))
    plt.title(f"Image: {os.path.basename(image_path)}\nTrue: {true_label_text}\nPrediction ({model_str}): {prediction_text}")
    plt.axis('off')
    plt.show()

def reproduce_original_test_results(artifacts, model_str, dataset_root):
    print(f"\n--- Reproducing Original Test Set Results using train_test_split for {model_str} ---")

    model = artifacts.get(model_str)
    if model is None:
        print(f"Model {model_str} not found. Cannot reproduce original test results.")
        return

    X_sample_test, y_sample_test = fetch_original_test_split(
        dataset_root,
        target_size=(128, 128),
        test_size=0.2,
        random_state=42
    )

    if X_sample_test is None or X_sample_test.size == 0:
        print("No test data available for reproduction.")
        return

    print(f"\n--- Evaluating {model_str} on recreated original test data ({X_sample_test.shape[0]} samples) ---")
    try:
        # Düzeltme: mlp_model için de girdi düzleştirildi
        if model_str == 'mlp_model':
            # Keras MLP, düzleştirilmiş girdi bekler
            flat_features_test = X_sample_test.reshape(X_sample_test.shape[0], -1) # Şekil (N, H*W*C)
            prediction_proba = model.predict(flat_features_test, verbose=0)
            y_pred = (prediction_proba > 0.5).astype(int).flatten()
        else: # Sklearn modelleri (N, num_features) bekler
            flat_features_test = X_sample_test.reshape(X_sample_test.shape[0], -1) # Şekil (N, H*W*C)
            if hasattr(model, 'predict_proba'):
                prediction_proba = model.predict_proba(flat_features_test)
                y_pred = (prediction_proba[:, 1] > 0.5).astype(int)
            else:
                y_pred = model.predict(flat_features_test)

        accuracy = accuracy_score(y_sample_test, y_pred)
        f1 = f1_score(y_sample_test, y_pred, zero_division=0)
        print(f" - Accuracy: {accuracy:.4f}, F1: {f1:.4f}")
    except Exception as e:
        print(f"Error during evaluation for {model_str} on original test data: {e}")
        pass

dataset_choice = 'kaggle'
artifacts = load_all_artifacts(dataset_choice)

model_display_names = {
    'mlp_model': 'Keras MLP Model',
    'lightgbm_model': 'LightGBM Model',
    'svm_model': 'SVM Model',
    'transformer_lightgbm_model': 'LGBM Transformer Model'
}

for model_key, display_name in model_display_names.items():
    current_model = artifacts.get(model_key)
    if current_model is None:
        print(f"\n--- Skipping {display_name} as it was not loaded ---")
        continue

    print(f"\n--- Running evaluations for: {display_name} ({model_key}) ---")

    reproduce_original_test_results(artifacts, model_key, DATA_ROOT_KAGGLE)

    evaluate_folder(TEST_EVAL_DATA_ROOT, artifacts, model_key)

    fire_images_in_test_folder = [f for f in os.listdir(os.path.join(TEST_EVAL_DATA_ROOT, 'fire_images')) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    non_fire_images_in_test_folder = [f for f in os.listdir(os.path.join(TEST_EVAL_DATA_ROOT, 'non_fire_images')) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if fire_images_in_test_folder:
        for i in range(min(len(fire_images_in_test_folder), 3)):
            sample_fire_image = os.path.join(TEST_EVAL_DATA_ROOT, 'fire_images', fire_images_in_test_folder[i])
            process_single_image(sample_fire_image, artifacts, model_key)
    else:
        print(f"\nNo fire images found in {os.path.join(TEST_EVAL_DATA_ROOT, 'fire_images')}. Skipping single fire image test.")

    if non_fire_images_in_test_folder:
        for i in range(min(len(non_fire_images_in_test_folder), 3)):
            sample_non_fire_image = os.path.join(TEST_EVAL_DATA_ROOT, 'non_fire_images', non_fire_images_in_test_folder[i])
            process_single_image(sample_non_fire_image, artifacts, model_key)
    else:
        print(f"\nNo non-fire images found in {os.path.join(TEST_EVAL_DATA_ROOT, 'non_images')}. Skipping single non-fire image test.")


--- Loading saved models for kaggle dataset ---
Loaded mlp_model from ..\..\models\Dfire_hybrid_custom_mlp_best_model_Scaled_All_CNN.keras
Could not load mlp_model or create CNN feature extractor for MLP: name 'create_custom_cnn' is not defined
Loaded transformer_selector (SelectKBest) from ..\..\models\Dfire_hybrid_transformer_Scaled_Corr75%_CNN.pkl
Could not load transformer_lightgbm_model components: [Errno 2] No such file or directory: '..\\..\\models\\Dfire_hybrid_transformer_Scaled_Corr75%_CNN_LGBM.pkl'

--- Running evaluations for: Keras MLP Model (mlp_model) ---

--- Reproducing Original Test Set Results using train_test_split for mlp_model ---

--- Fetching original test split from '..\..\data_subsets\fire_dataset' ---
Successfully recreated test split with 200 samples.

--- Evaluating mlp_model on recreated original test data (200 samples) ---
Error during evaluation for mlp_model on original test data: Exception encountered when calling Sequential.call().

Input 0 of layer 

Processing folder images:   0%|          | 0/200 [00:00<?, ?it/s]

MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\fire_images\1.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\fire_images\10.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\fire_images\100.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\fire_images\11.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\fire_images\12.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\fire_images\13.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\fire_images\14.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\fire_images\15.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\fire_images\16.jpg. Skipping.
M

Processing folder images:  12%|█▏        | 23/200 [00:00<00:00, 224.40it/s]

MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\fire_images\3.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\fire_images\30.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\fire_images\31.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\fire_images\32.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\fire_images\33.png. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\fire_images\34.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\fire_images\35.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\fire_images\36.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\fire_images\37.jpg. Skipping.
ML

...

Processing folder images:  87%|████████▋ | 174/200 [00:00<00:00, 201.40it/s]

MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\non_fire_images\313.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\non_fire_images\317.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\non_fire_images\318.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\non_fire_images\327.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\non_fire_images\329.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\non_fire_images\338.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\non_fire_images\339.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\non_fire_images\350.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\tes

Processing folder images: 100%|██████████| 200/200 [00:01<00:00, 175.31it/s]

MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\non_fire_images\422.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\non_fire_images\423.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\non_fire_images\425.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\non_fire_images\435.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\non_fire_images\436.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\non_fire_images\438.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\non_fire_images\447.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test_kaggle\non_fire_images\49.jpg. Skipping.
MLP model or CNN feature extractor not loaded for ..\..\data_subsets\test

Error during prediction for ..\..\data_subsets\test_kaggle\fire_images\100.jpg with mlp_model: Exception encountered when calling Sequential.call().

Input 0 of layer "dense_62" is incompatible with the layer: expected axis -1 of input shape to have value 65536, but received input with shape (1, 49152)

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 49152), dtype=float32)
  • training=False
  • mask=None

--- Processing single image: ..\..\data_subsets\test_kaggle\non_fire_images\1.jpg using mlp_model ---
Error during prediction for ..\..\data_subsets\test_kaggle\non_fire_images\1.jpg with mlp_model: Exception encountered when calling Sequential.call().

Input 0 of layer "dense_62" is incompatible with the layer: expected axis -1 of input shape to have value 65536, but received input with shape (1, 49152)

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 49152), dtype=float32)
  • training=False
  • mask=None

--- Processing single image: